# Imports

In [ ]:
import itertools as it
from pathlib import Path

import dask
import distributed
import h5py
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
from cytoolz import partial
from dask_jobqueue import SLURMCluster
from distributed import Client
from IPython.display import Video
from PIL import Image, ImageDraw, ImageFont
from tqdm.auto import tqdm, trange

In [ ]:
from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar.register()

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

In [ ]:
from paulssonlab.image_analysis import mosaic, workflow

# Config

In [ ]:
# nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/220704/220704rbs_library_fish.nd2"
nd2_filename = "/home/jqs1/scratch/jqs1/microscopy/220718/RBS_DEG_library_20x.nd2"

In [ ]:
nd2 = nd2reader.ND2Reader(nd2_filename)

In [ ]:
default_channels = ("YFP-DUAL", "RFP-Penta")
# default_channels = ("YFP-DUAL", "RFP-PENTA")
channel_to_color = {
    "BF": "#ffffff",
    "RFP-PENTA": "#e22400",
    "RFP-Penta": "#e22400",
    "YFP-DUAL": "#f5eb00",
    # "GFP": "#76ba40",
    "Cy5": "#e292fe",
    # "Cy7": "#FF0000"
    # "BFP": "#3a87fd",
}

In [ ]:
font = ImageFont.truetype("fira/FiraSans-Medium.ttf")

In [ ]:
dask.config.set({"distributed.scheduler.allowed-failures": 10})

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="4GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster.scale(50)

In [ ]:
cluster.adapt(maximum=300)

In [ ]:
cluster

# Mosaic

In [ ]:
%%time
extrema = mosaic.get_intensity_extrema(nd2, ("YFP-DUAL", "RFP-Penta"))

In [ ]:
extrema

In [ ]:
# scaling_funcs = mosaic.get_scaling_funcs(extrema)
# scaling_funcs = mosaic.get_scaling_funcs({"YFP-DUAL": (262, 8000), "RFP-Penta": (278, 8000)})
scaling_funcs = mosaic.get_scaling_funcs(
    {"YFP-DUAL": (300, 6000), "RFP-Penta": (1200, 7000)}
)

In [ ]:
def positions_func(positions):
    return positions.drop(["x_idx", "y_idx"], axis=1).join(
        positions["position_name"].apply(
            lambda name: pd.Series(
                [int(idx) for idx in name.split(".")], index=["y_idx", "x_idx"]
            )
        )
    )

In [ ]:
%%time
num_t = nd2.sizes["t"]
# scale = [80,40,20,10,5,1]#it.repeat(80)
# scale = np.geomspace(80, 0.1, 900)
# scale = np.geomspace(0.5, 0.1, 60)
# scale = [40]
scale = [0.1]  # [0.3]
# scale = [0.333]
# scale = it.repeat(0.1)
# scale = it.repeat(0.4)
# scale = np.geomspace(20, 0.05, num_t*3)# + np.linspace(0, 0, 0) #it.repeat(0.3)  # [0.3, 0.3, 0.3, 0.3]
# timepoints = range(0, 119, 30)
# timepoints = range(0, 110, 10)#[20,40,60]
timepoints = [0]  # it.repeat(60)  # [20]  # [20,40,60,80]
# timepoints = range(num_t)
# timepoints = it.chain(range(num_t), range(num_t), range(num_t)) #[20]#[20, 40, 60]
offset = [2000, 1000]  # [0,0]#np.array([604, 354])
animation_delayed = mosaic.mosaic_animate_scale(
    nd2_filename,
    scale,
    timepoints=timepoints,
    scaling_funcs=scaling_funcs,
    # offset=offset,
    rotation=np.deg2rad(-0.65),
    channels=default_channels,
    channel_to_color=channel_to_color,
    # overlay_func=partial(
    #     mosaic.square_overlay,
    #     min_scale=80,
    #     min_n=0,
    #     min_width=0.5,
    #     max_scale=0.1,
    #     max_n=5,
    #     max_width=0.9,
    #     font=font,
    # ),
    positions_func=positions_func,
    output_dims=(3840, 2160),
    delayed=True,
)

In [ ]:
%%time
a = dask.compute(animation_delayed, scheduler="sync")[0]

In [ ]:
plt.figure(figsize=(40, 40))
plt.imshow(a[0])

In [ ]:
plt.figure(figsize=(40, 40))
plt.imshow(a[0])

In [ ]:
plt.figure(figsize=(40, 40))
plt.imshow(a[0])

In [ ]:
plt.figure(figsize=(40, 40))
plt.imshow(a[0])

In [ ]:
plt.figure(figsize=(40, 40))
plt.imshow(a[0])

In [ ]:
plt.figure(figsize=(40, 40))
plt.imshow(a[0])

In [ ]:
%%time
# animation_future = client.compute(animation_delayed)
animation_future = [client.compute(a) for a in tqdm(animation_delayed)]

In [ ]:
%%time
a = client.gather(animation_future)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(a[-1])

In [ ]:
import cv2

In [ ]:
x = a[0].copy()

In [ ]:
ft = cv2.freetype.createFreeType2()
ft.loadFontData(fontFileName="fira/FiraSans-Medium.ttf", id=0)
# y = np.zeros_like(x, dtype=np.uint8)
y = np.zeros((800, 600, 3), dtype=np.uint8)
ft.putText(
    img=y,
    text="abc",
    org=(15, 15),
    fontHeight=101,
    color=(255, 100, 200),
    thickness=-1,
    line_type=cv2.LINE_AA,
    bottomLeftOrigin=False,
)
plt.imshow(y)

In [ ]:
np.unique(y[:, :, 0] - y[:, :, 1])

In [ ]:
y[:, :, 1]

In [ ]:
plt.scatter(y[:, :, 0], y[:, :, 1])

In [ ]:
np.unique(y)

In [ ]:
x.shape

In [ ]:
cv2.putText(x, "", "")

In [ ]:
import cairo

In [ ]:
data = numpy.ndarray(shape=(height, width), dtype=numpy.uint32)
surface = cairo.ImageSurface.create_for_data(data, cairo.FORMAT_ARGB32, width, height)
cr = cairo.Context(surface)

cr.scale(200, 100)
cr.set_line_width(0.04)

cr.select_font_face("Sans", cairo.FONT_SLANT_NORMAL, cairo.FONT_WEIGHT_BOLD)
cr.set_font_size(0.35)

cr.move_to(0.04, 0.53)
cr.show_text("Hello")

cr.move_to(0.27, 0.65)
cr.text_path("void")
cr.set_source_rgb(0.5, 0.5, 1)
cr.fill_preserve()
cr.set_source_rgb(0, 0, 0)
cr.set_line_width(0.01)
cr.stroke()

# draw helping lines
cr.set_source_rgba(1, 0.2, 0.2, 0.6)
cr.arc(0.04, 0.53, 0.02, 0, 2 * np.pi)
cr.arc(0.27, 0.65, 0.02, 0, 2 * np.pi)
cr.fill()

In [ ]:
?surface.map_to_image

In [ ]:
z

In [ ]:
np.asarray(z)

In [ ]:
plt.imshow(z)

In [ ]:
x = np.zeros((1024, 512, 3))
y = mosaic.square_overlay(x, 1, 8.5, font=font)
plt.figure(figsize=(20, 20))
plt.imshow(y)

In [ ]:
%%time
mosaic.export_video(
    a,
    "/home/jqs1/scratch/jqs1/microscopy/220704/mosaics/timelapse_zoomtest_4.mp4",
    fps=10,
)

In [ ]:
Video(
    "/home/jqs1/scratch/jqs1/microscopy/220704/mosaics/timelapse_3x3_5fps.mp4",
    embed=True,
)

# FISH

In [ ]:
fish_dir = Path("/home/jqs1/scratch/jqs1/microscopy/220718/FISH/real_run")

In [ ]:
def get_fish_frame(filename, v, channel, t):
    with h5py.File(filename / f"fov={v}_config={channel}_t={t}") as f:
        frame = f["data"][()]
    return frame

In [ ]:
fish_colors = {
    "BF": "#ffffff",
    "RFP": "#e22400",
    # "YFP-DUAL": "#f5eb00",
    # "GFP": "#76ba40",
    "Cy5": "#e292fe",
    "Cy7": "#00faff"
    # "BFP": "#3a87fd",
}

In [ ]:
fish_scaling_funcs = get_scaling_funcs(
    {
        "BF": (40_000, 65_500),
        "RFP": (8000, 15000),
        "Cy5": (5_000, 40000),
        "Cy7": (2500, 4_000),
    }
)

In [ ]:
fish_scaling_funcs["Cy7"](2172)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["Cy5"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["RFP"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["RFP", "Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["BF", "RFP", "Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
a = colorized_frame(
    fish_colors,
    get_fish_frame,
    fish_dir,
    1,
    8,
    ["BF", "RFP", "Cy5", "Cy7"],
    scaling_funcs=fish_scaling_funcs,
)
plt.figure(figsize=(40, 40))
plt.imshow(a)

In [ ]:
%%time
frames = [
    colorized_frame(
        fish_colors,
        get_fish_frame,
        fish_dir,
        t,
        8,
        ["Cy7"],
        scaling_funcs=fish_scaling_funcs,
    )
    for t in trange(1, 11)
]

In [ ]:
%%time
rescaled_frames = [
    skimage.transform.rescale(f, 0.5, anti_aliasing=True, channel_axis=-1)
    for f in tqdm(frames)
]

In [ ]:
%%time
export_video(rescaled_frames, "/home/jqs1/_temp/FISH_Cy7.mp4", fps=5)

In [ ]:
!du -hs /home/jqs1/_temp/FISH_BF.mp4

# Grid debugging

In [ ]:
from paulssonlab.image_analysis.workflow import (
    get_filename_image_limits,
    get_nd2_frame,
    get_position_metadata,
    parse_nd2_metadata,
)

In [ ]:
nd2 = nd2reader.ND2Reader(nd2_filename)
nd2s = {filename: nd2 for filename in (nd2_filename,)}
metadata = {nd2_filename: parse_nd2_metadata(nd2) for nd2_filename, nd2 in nd2s.items()}
positions = get_position_metadata(metadata)

In [ ]:
positions = positions_func(positions)

In [ ]:
positions

In [ ]:
first_row = positions[positions["y_idx"] == positions["y_idx"].min()]

In [ ]:
first_row

In [ ]:
upper_left = first_row[first_row["x_idx"] == first_row["x_idx"].min()].squeeze()

In [ ]:
upper_right = first_row[first_row["x_idx"] == first_row["x_idx"].max()].squeeze()

In [ ]:
upper_right["y"]

In [ ]:
z = np.arctan2(upper_right["y"] - upper_left["y"], upper_right["x"] - upper_left["x"])

In [ ]:
np.rad2deg(
    np.arctan2(upper_left["y"] - upper_right["y"], upper_left["x"] - upper_right["x"])
)

In [ ]:
positions

In [ ]:
(47486.679688 - 65486.679688) * np.tan(np.deg2rad(0.6))

In [ ]:
# y
54474.778675 - 63727.679688

In [ ]:
# x
(47486.679688 - 65486.679688)

In [ ]:
9252 / 18000

In [ ]:
np.arctan(0.514)

In [ ]:
np.arc